# Проект. Защита данных страховой компании

Имеется задача защитить данные клиентов страховой компании «Хоть потоп». В ходе проекта необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию и обосновать корректность работы метода.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

------

## 1. Загрузка данных

Импортируем необходимые модули и библиотеки.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from numpy.linalg import inv
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

Прочитаем данные и сохраним их в переменную `data`.

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

Изучим информацию о данных.

In [3]:
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Имеется таблица, у которой 5 столбцов и 5000 строк. Пропусков в данных нет. Имеется 4 признака: *'Пол'*, *'Возраст'*, *'Зарплата'*, *'Члены семьи'* и целевой признак - *'Страховые выплаты'*. Целевой признак - количество страховых выплат клиенту за последние 5 лет, поэтому это задача регрессии. Аномалий-выбросов в данных не наблюдается.

У значений столбцов *'Возраст'* и *'Зарплата'* поменяем тип данных на целочисленный.

In [6]:
data['Возраст'] = data['Возраст'].astype('int')
data['Зарплата'] = data['Зарплата'].astype('int')

------

## 2. Умножение матриц

**Вопрос по проекту:** признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** не изменится.

**Обоснование:** качество линейной регрессии не изменится, так как каждый элемент преобразованной матрицы равен скалярному произведению строки исходной матрицы на соответствующий столбец обратимой матрицы, таким образом, значения вектора признаков изменяются одинаково пропорционально, такое же преобразование происходит при выполнении алгоритма линейной регрессии, например, при первой итерации, когда признаки умножаются на некоторые случайные веса. Таким образом, параметры линейной регрессии в исходной задаче и в преобразованной будут отличаться  вектором весов $w$, а сдвиг $w_{0}$ останется прежним. Значения вектора весов $w$ в преобразованной задаче будут пропорциональны значениям в исходной задаче в зависимости от значений обратимой матрицы.

Условие обратимости матрицы, на которую происходит умножение, позволяет при необходимости получить начальные значения признаков, умножив преобразованные признаки на обратную матрицу.

------

## 3. Алгоритм преобразования

Запишем алгоритм преобразования.

**Алгоритм**

$$X_{trans} = X\bullet A$$

$$A = \frac{P + P1}{P\bullet P1}\$$

Обозначения:

- $X$ — матрица признаков

- $X_{trans}$ — трансформированная матрица признаков

- $A$ — обратимая матрица, на которую умножаются признаки

- $P, P1$ — "случайные" обратимые матрицы

**Обоснование**

В результате операций с матрицами $P$ и $P1$ получается обратимая матрица $A$. Как было обосновано выше, при умножении признаков на обратимую матрицу качество линейной регрессии не изменится.

------

## 4. Проверка алгоритма

Напишем функцию `check_inv`, которая будет проверять матрицу на обратимость.

In [7]:
def check_inv(X):
    try:                 # для проверки используем обработчик исключений
        inv(X)           # пытаемся найти обратную матрицу   
        return 'True'    # если ошибки нет, выводим 'True'
    except:
        return 'False'   # если исходная не имеет обратную матрицу, выводим 'False'

Напишем функцию `transformation`, которая будет реализовывать алгоритм из пункта 3 проекта.

In [8]:
def transformation(data):                         # функция принимает на вход признаки
    A = (P + P1) / np.dot(P, P1)                  # формула для преобразования матриц P и P1
    if check_inv(A) == 'True':                    # проверяем обратима ли матрица
        return np.dot(np.array(data), A)          # если да, то выводим результат
    else:
        print('Проверьте обратимость матриц')     # в противном случае выводим сообщение с требованием проверки
                                                  # условия обратимости матрицы

Приступим к проверке качества линейной регрессии на исходных данных и на преобразованных. Раделим исходный датасет не признаки и целевой признак.

In [9]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

Разобьем признаки на обучающую и проверочную выборки.

In [10]:
features_train, features_valid, target_train, target_valid = train_test_split(
features, target, test_size=0.25, random_state=12345)

Так как в данном проекте не нужно добиваться от модели лучшего качества, то пропустим этап стандартизации значений признаков.
Обучим модель линейной регрессии на исходных данных и выведем на экран метрику R2.

In [11]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print('R2_score = {:.6f}'.format(r2_score(target_valid, predictions)))

R2_score = 0.435228


Теперь займемся преобразование данных и обучением на них модели линейной регрессии.

Сформируем случайные матрицы $P$ и $P1$ размером 4 на 4.

In [12]:
P = np.random.random((4,4))
P1 = np.random.random((4,4))

Закомментированный код ниже использовался для проверки работы функций `transformation` `check_inv` в случае, если матрицы не имеют обратных матриц.

In [13]:
#P = np.zeros((4,4))
#P += np.arange(4)
#P1 = P

Преобразуем признаки `features_train` и `features_valid` по нашему алгоритму.

In [14]:
features_train_trans = transformation(features_train)

In [15]:
features_valid_trans = transformation(features_valid)

Обучим модель линейной регрессии на преобразованных данных и выведем на экран значение метрики R2.

In [16]:
model_trans = LinearRegression()
model_trans.fit(features_train_trans, target_train)
predictions = model_trans.predict(features_valid_trans)
print('R2_score = {:.6f}'.format(r2_score(target_valid, predictions)))

R2_score = 0.435228


------

## Выводы

У моделей, обученных на исходных и на преобразованных данных метрика R2 совпадает до 6 знака после запятой. Это свидетельствует о том, что наш алгоритм преобразования работает: надежно защищает данные и в то же время, не уменьшает качество моделей, которые обучаются на преобразованных данных.

Если возникнет необходимость расшифровать данные, понадобятся сформированные матрицы $P$ и $P1$ и формула преобразования, поэтому их нужно сохранять отдельно.

------